In [1]:
from copy import copy
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.collection.prov_execution.execution import NotebookQuerierOptions
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier import QuerierOptions


import numpy as np
#import ipdb

In [2]:
import numpy as np

# Helper functions
def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())


# Teste

In [3]:
x = 1
y = now_variable('y', x)

Evaluation(id=20, checkpoint=4.629562253, code_component_id=247, activation_id=14, repr=1)


In [4]:
q = 100

In [5]:
z = y + q

In [6]:
w = now_variable('w', z)

Evaluation(id=34, checkpoint=4.813965184000001, code_component_id=272, activation_id=31, repr=101)


In [7]:
eval_y = get_evaluation(20)

In [9]:
eval_w = get_evaluation(34)

In [10]:
list(eval_w.dependencies_as_dependent)

[dependency(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 31, 34, 26, 30, 'assignment', 1, nil, nil, nil).]

In [11]:
list(eval_w.dependencies_as_dependency)

[dependency(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 31, 35, 31, 34, 'argument', 1, nil, nil, nil).,
 dependency(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 31, 35, 31, 34, 'dependency', 1, nil, nil, nil).]

### Investigating deps

In [12]:
for i in list(eval_y.dependencies_as_dependency):
    print(i.dependent.code_component)

code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 241, 'now_variable(''y'', x)', 'call', 'r', 2, 4, 2, 24, 233).
code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 241, 'now_variable(''y'', x)', 'call', 'r', 2, 4, 2, 24, 233).


In [13]:
qops = QuerierOptions()
nbqops = NotebookQuerierOptions(level=True)
for evaluation in [eval_y]:
    context = NodeContext(evaluation, None, options=nbqops)

In [14]:
for i in context.dependencies_backward():
    print(i.evaluation.code_component)

code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 236, 'x', 'name', 'w', 1, 0, 1, 1, 233).
code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 232, '<class ''int''>', 'type', 'w', -1, -1, -1, -1, nil).


In [15]:
for i in context.dependencies_forward():
    print(i.evaluation.code_component)

code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 247, 'x', 'name', 'r', 2, 22, 2, 23, 233).
code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 247, 'x', 'name', 'r', 2, 22, 2, 23, 233).
code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 232, '<class ''int''>', 'type', 'w', -1, -1, -1, -1, nil).


#### Testing with querier

In [17]:
class NotebookQuerierOptions(QuerierOptions):
    global body_function_def
    dep_list = []
    
    
    def __init__(self, level, *args, **kwargs):
        QuerierOptions.__init__(self, *args, **kwargs) # change it to super when integrating in the main code
        self.level = level
    
    def visit_arrow_forward(self, context, neighbor):
        print('context', context.evaluation.code_component)
        print('nieghbor', neighbor.evaluation.code_component)
    
    def visit_arrow_backward(self, context, neighbor):
                
        # keeping 
        if neighbor.evaluation.code_component.type == 'function_def':
            body_function_def.append(int(neighbor.evaluation.code_component.id))
       
        arrow_list = ('argument', '<.__class__>', 'item')
        type_list = ('add', 'literal', 'mult', 'div', 'param', 'sub', 'attribute', 'usub', 'function_def')
        
        context_code_comp = context.evaluation.code_component
        neighbor_code_comp = neighbor.evaluation.code_component
           
        if neighbor.arrow not in arrow_list:
            if context_code_comp.type not in type_list:
                if neighbor_code_comp.type not in type_list:
                    if not (neighbor.arrow == 'use' and context_code_comp.type == 'call'):
                        if (neighbor_code_comp.container_id != None):
                            if (neighbor_code_comp.container_id not in body_function_def) or self.level:
                                self.dep_list.append((str(context.evaluation.checkpoint), str(context.evaluation.id), context_code_comp.name, context.evaluation.repr))

    def public_print(self):
        global dep_dict
        
        dep_dict = {i[0] : i[1] for i in enumerate(self.dep_list)}
        return dep_dict
        
                                
body_function_def = []
dep_dict = {}

nbOptions = NotebookQuerierOptions(level=False)
querier = DependencyQuerier(options=nbOptions)
nodes_to_visit, visited, found = querier.navigate_dependencies([eval_y])  

context code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 247, 'x', 'name', 'r', 2, 22, 2, 23, 233).
nieghbor code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 232, '<class ''int''>', 'type', 'w', -1, -1, -1, -1, nil).
context code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 232, '<class ''int''>', 'type', 'w', -1, -1, -1, -1, nil).
nieghbor code_component(98e2f930-79b7-4c31-ab24-3fdddacd79e3, 120, '<class ''type''>', 'type', 'w', -1, -1, -1, -1, nil).
